In [3]:
%%HTML
<script src="require.js"></script>

In [4]:
from IPython.display import HTML
HTML('''<script src="https://cdnjs.cloudflare.com/ajax/libs/jquery/2.0.3/jquery.min.js "></script><script>
code_show=true; 
function code_toggle() {
if (code_show){
$('div.jp-CodeCell > div.jp-Cell-inputWrapper').hide();
} else {
$('div.jp-CodeCell > div.jp-Cell-inputWrapper').show();
}
code_show = !code_show
} 
$( document ).ready(code_toggle);</script><form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>
''')

<div align="center">
<img src="Figures/Title.png" />
</div>

<div style="background-color: #270f47; color: #eed47c; padding: 15px; text-align: left; border-radius: 5px;">

# I. Table of Contents

</div>

<div style="background-color: #270f47; color: #eed47c; padding: 15px; text-align: left; border-radius: 5px;">

# II. Introduction

</div>

<div style="color: #270f47; padding: 15px; text-align: left; border-radius: 5px;">

## Problem Statement

</div>

<div style="color: #270f47; padding: 15px; text-align: left; border-radius: 5px;">

## Motivation

</div>

<div style="background-color: #270f47; color: #eed47c; padding: 15px; text-align: left; border-radius: 5px;">

# III. Methodology Overview

</div>

<div align="center">
<img src="Figures/Methodology Overview.png" />
</div>

The figure above shows the methodology overview of this study.
Our methodology consists of a four step process starting with the collection of historical Binance data, followed by exploratory data analysis, data processing, and the results and discussion. 
Under the results and discussion portion, we would also discuss the potential impacts this can have on investors, the perceived benefits and disadvantages, our recommendations and conclusions. 

Below would be a short summary of each of the different steps:
1. **Data Collection**: Here, we collect publicly available historical Binance data focusing on the closing price. Since the dataset is very big, we stored it in a Spark DataFrame.
2. **Exploratory Data Analysis**: Here, we prepared and cleaned the dataset to ensure that it is suitable for further analysis. EDA is used to understand the behavior of the closing price for different classifications: base currencies, quote currencies, and even their classifications. These would be further discussed later on.
3. **Data Processing**: In this step, we want to get answers for our research problem which is to find out the volatility, price stability, and trading activity patterns among the three classifications: cryptocurrency, fiat-based tokens, and stablecoin.
4. **Results and Discussion**: Based on what we find, we want to discuss what this could mean for investors, mention our perceived benefits and disadvantages, and recommend further steps. Afterwards, we will summarize findings in the recommendations and conclusion section.

<div style="background-color: #270f47; color: #eed47c; padding: 15px; text-align: left; border-radius: 5px;">

# IV. Data Collection

</div>

<div style="color: #270f47; padding: 15px; text-align: left; border-radius: 5px;">

## Data Source

</div>

We gathered data from the Binance Full History dataset available on the Jojie-collected public datasets of the Asian Institute of Management. 
The directory is as follows: `/mnt/data/public/binance-full-history`.
This dataset consists of more than 30 Parquet files that correspond to the information per base currency in its quote currency.
The dataset, as a whole, takes up 33 GB of data, so it would not be feasible to use the typical Python operations to handle all of it. 
Instead, we used Apache Spark, and created a Spark DataFrame just to contain all the information we need.

<div style="color: #270f47; padding: 15px; text-align: left; border-radius: 5px;">

## Data Description

</div>

The following features are consistent within all Parquet files in the Binance Full History dataset:

| Feature Name | Data Description | Data Type |
|-------------|-----------------|-----------|
| open | Opening price of the trading period | float |
| high | Highest price during the trading period | float |
| low | Lowest price during the trading period | float |
| close | Closing price of the trading period | float |
| volume | Total volume of base asset traded | float |
| quote_asset_volume | Total volume of quote asset traded | float |
| number_of_trades | Total number of trades executed | integer |
| taker_buy_base_asset_volume | Volume of base asset bought by takers | float |
| taker_buy_quote_asset_volume | Volume of quote asset bought by takers | float |
| open_time | Timestamp when the trading period started | timestamp_ntz |

<div style="color: #270f47; padding: 15px; text-align: left; border-radius: 5px;">

## Data Collection and DataFrame Creation

</div>

Since there are a lot of files, and the entire dataset sums to a total of 33 GB, we had to use Apache Spark to handle the data. 
To summarize the code below, we created a Spark DataFrame from all the information available.
Here, we created new features: `base_currency`, `quote_currency`, and `classification`. 
These three are not anywhere in the previously shown data dictionary because they are information you can only get from the file names of the Parquet files.

In [6]:
# standard imports
import matplotlib.pyplot as plt
import random
import pandas as pd
import glob
import os
import numpy as np
from functools import reduce

# spark imports
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import FloatType
import pyspark.sql.window as window
from pyspark.sql.functions import col, split, when, lit
from pyspark.sql.functions import regexp_extract, input_file_name

ModuleNotFoundError: No module named 'pyspark'

In [ ]:
# Directory containing the parquet files
directory = '/mnt/data/public/binance-full-history'
files = [f for f in os.listdir(directory) if f.endswith('.parquet')]

base_quote_counts = {}

for file in files:
    # Get base and quote per file name
    base_currency = file.split('-')[0]
    quote_currency = file.split('-')[1].replace('.parquet', '')
    
    if base_currency not in base_quote_counts:
        base_quote_counts[base_currency] = set()
    
    base_quote_counts[base_currency].add(quote_currency)

# Convert to counts
base_counts = {base: len(quotes) for base, quotes in base_quote_counts.items()}

# Create dataframe for plotting
counts_df = pd.DataFrame({
    'base_currency': list(base_counts.keys()),
    'quote_count': list(base_counts.values())
})
quote_count_distribution = counts_df['quote_count'].value_counts().sort_index()

# Plotting
plt.figure(figsize=(15, 8))
bars = plt.bar(
    quote_count_distribution.index, quote_count_distribution.values
)

for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.1,
             f'{int(height)}', ha='center', va='bottom')

plt.xlabel('Number of Quote Currencies')
plt.ylabel('Number of Base Currencies')
plt.title('Distribution of Base Currencies by Number of Quote Currencies')
plt.grid(axis='y', alpha=0.3)
plt.xticks(quote_count_distribution.index)

# Saving the figure
plt.tight_layout()
plt.savefig(
    'Figures/Distribution of Base Currencies by Number of Quote Currencies.png'
)
plt.show()

# Get top base currencies
top_bases = counts_df.nlargest(10, 'quote_count')
common_quotes = set.intersection(*(base_quote_counts[base] for base in top_bases['base_currency']))

As seen here, there are a lot of files, but most of them only have one quote currency.
Furthermore, there are also not a lot of base currencies that have a lot of the same quote currencies.
Since we want to do a comparative analysis on the different quote currencies and their relationship to the base currency, we will only look at the top 10 base currencies so that we have files that we can compare with others.
Additionally, we will also only look at the same quote currencices that are across all the top 10 base currencies.
This cuts down on a lot of irrelevant data that are not important to our analysis and makes this more computationally efficient. 

In [ ]:
# Initialize Spark Session
spark = SparkSession.builder.appName("TopBasesAnalysis").getOrCreate()
directory = '/mnt/data/public/binance-full-history'

# Getting only target files
file_data = []
for base in top_bases['base_currency']:
    for quote in common_quotes:
        file_name = f"{base}-{quote}.parquet"
        file_path = os.path.join(directory, file_name)
        if os.path.exists(file_path):
            file_data.append((file_path, base, quote))

# Define classification mapping
classification_map = {
    'stablecoin': ['BUSD', 'USDT'],
    'fiat': ['EUR']
}

# Create DataFrames for each file and add columns
dfs = []
for file_path, base, quote in file_data:
    df = spark.read.parquet(file_path)
    df = df.withColumn("base_currency", lit(base)) \
           .withColumn("quote_currency", lit(quote)) \
           .withColumn("classification", 
                      when(lit(quote).isin(classification_map['stablecoin']), lit('stablecoin'))
                      .when(lit(quote).isin(classification_map['fiat']), lit('fiat'))
                      .otherwise(lit('cryptocurrency')))
    dfs.append(df)

# Union all DataFrames
spark_df = reduce(lambda df1, df2: df1.union(df2), dfs)

The schema, also features, of the new Spark DataFrame is as follows:

| Feature | Description | Data Type |
|---------|-------------|-----------|
| open | Opening price of the trading period | float |
| high | Highest price during the trading period | float |
| low | Lowest price during the trading period | float |
| close | Closing price of the trading period | float |
| volume | Total trading volume in base currency | float |
| quote_asset_volume | Total trading volume in quote currency | float |
| number_of_trades | Total number of trades executed during the period | integer |
| taker_buy_base_asset_volume | Volume of base asset bought by takers (market orders) | float |
| taker_buy_quote_asset_volume | Volume of quote asset bought by takers (market orders) | float |
| open_time | Timestamp indicating the start of the trading period | timestamp_ntz |
| base_currency | The base currency in the trading pair (e.g., BTC, ETH) | string |
| quote_currency | The quote currency in the trading pair (e.g., USDT, BUSD) | string |
| classification | Classification of the quote currency (stablecoin/fiat/cryptocurrency) | string |

<div style="background-color: #270f47; color: #eed47c; padding: 15px; text-align: left; border-radius: 5px;">

# V. Exploratory Data Analysis

</div>

<div style="background-color: #270f47; color: #eed47c; padding: 15px; text-align: left; border-radius: 5px;">

# VI. Data Processing

</div>

In this portion, we are trying to fulfill our goals where we compare volatility, price stability, and trading activity patterns.

<div style="background-color: #270f47; color: #eed47c; padding: 15px; text-align: left; border-radius: 5px;">

# VII. Results and Discussion

</div>

<div style="background-color: #270f47; color: #eed47c; padding: 15px; text-align: left; border-radius: 5px;">

# VIII. Conclusion and Recommendations

</div>

<div style="color: #270f47; padding: 15px; text-align: left; border-radius: 5px;">

## Conclusion

</div>

<div style="color: #270f47; padding: 15px; text-align: left; border-radius: 5px;">

## Recommendations

</div>

### Limitations of the Study

insert your text here

### Recommendations for Readers

insert your text here

### Future Work

insert your text here

<div style="background-color: #270f47; color: #eed47c; padding: 15px; text-align: left; border-radius: 5px;">

# IX. References

</div>

<div style="background-color: #270f47; color: #eed47c; padding: 15px; text-align: left; border-radius: 5px;">

# X. Supplementary

</div>